In [2]:
import cv2; print('opencv version:', cv2.__version__)
import pandas as pd; print('panda version: ', pd.__version__)
import random;
from random import randint;
import os; 
import numpy as np;
from pathlib import Path
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt; 
from skimage.filters import threshold_otsu
import joblib
print(os.getcwd())

base_dir = os.getcwd() + "/.."
dirs_data = []

text = sorted(os.listdir(base_dir + '/data'))
random.seed(448)
random.shuffle(text)

for item in text:
    dirs_data.append([base_dir + '/data/' + item + '/plate.png',
                     base_dir + '/data/' + item + '/car.jpg',
                     item])
    my_file = Path(base_dir + '/data/' + item + '/plate.png')
    if not my_file.is_file():
        print("Error: " + item)
    my_file = Path(base_dir + '/data/' + item + '/car.jpg') 
    if not my_file.is_file():
        print("Error: " + item)

nPlates = 7000
nPlatesCount = 0
addedText = []
plateImg = []
binary = []

for _dir, _car, _text in dirs_data[0:nPlates:1]:
    img = cv2.imread(_dir,cv2.IMREAD_COLOR)
    height, width, channels = img.shape
    if(height == 50 and width == 235 and _text[0] != 'U'):
        nPlatesCount+=1
        addedText.append(_text)
        resizedImg = cv2.resize(img, (0,0), fx=0.5, fy=0.5)
        plateImg.append(cv2.cvtColor(resizedImg,cv2.COLOR_BGR2GRAY))
        binary.append([])
        
text = addedText
nPlates = nPlatesCount
testData = pd.DataFrame(
    {
        "text": addedText,
        "plate": plateImg,
        "plate_binary": binary
    }
)
testData.reset_index(drop=True, inplace=True)

for i in range(nPlatesCount):
    img_details = plateImg[i] < threshold_otsu(plateImg[i])
    testData.plate_binary[i] = img_details.reshape(-1)
    
image_data, target_data = testData.plate_binary, testData.text
target_data_only_first = []

for x in target_data:
    target_data_only_first.append(x);
    
image_data_norm = []
for x in image_data:
    image_data_norm.append(x);
        
image_data = image_data_norm
target_data = target_data_only_first

clf = svm.LinearSVC(penalty='l1', 
                    loss='squared_hinge', 
                    dual=False,
                    tol=1e-3)

clf.fit(image_data, target_data)

joblib.dump(clf, base_dir + "/models/model-l1-ta.svm")

opencv version: 3.4.2
panda version:  0.23.4
C:\Users\Rafal\Notebooks\Assignment6\Threading
Frequency of unique values of the said array:
[['00-01-TV' '00-15-HF' '00-36-PH' ... 'ZZ-95-93' 'ZZ-99-64' 'ZZ-99-81']
 ['1' '1' '1' ... '1' '1' '1']]


['C:\\Users\\Rafal\\Notebooks\\Assignment6\\Threading/../models/model-l1-ta.svm']